In [4]:
# Playfair cracker
import re
from functools import reduce

class Coords:
    def __init__(self, row, col):
        self.row = row
        self.col = col
        
    def __str__(self):
        return "({}, {})".format(self.col, self.row)
        
class Playfair:
    def __init__(self, key):
        if len(key) != 25:
            print("ERROR: Key must be 25 characters in length")
            return None
        
        self.keyMatrix = [[0 for x in range(5)] for y in range(5)]
        
        count = 0
        for a in key:
            col = int(count / 5)
            row = int(count % 5)
            self.keyMatrix[col][row] = a
            count += 1
            
    def printMatrix(self):
        for row in self.keyMatrix:
            print(row)
    
    def findCoord(self, letter):
        cols = [row.index(letter) if letter in row else -1 for row in self.keyMatrix]
        # Huge hack. All entries will be -1 except for one.
        # eg. [-1, -1, 2, -1, -1]
        # reduce() will sum all the entries, +4 will remove all of the -1s
        # eg. -1 + -1 + 2 + -1 + -1 + 4 = 2
        row = reduce(lambda x,y: x + y, cols) + 4
        # Huge hack part 2. Find the entry that is not -1
        col = list(map(lambda x: x == -1, cols)).index(False)
        
        return Coords(col, row)
    
    def encipher(self, digraph):
        a = digraph[0]
        b = digraph[1]
        coordA = self.findCoord(a)
        coordB = self.findCoord(b)
        newA = a
        newB = b
        
        if coordA.row == coordB.row:
            # Case 1: Same row
            newA = self.keyMatrix[coordA.row][(coordA.col + 1) % 5]
            newB = self.keyMatrix[coordB.row][(coordB.col + 1) % 5]
                    
        elif coordA.col == coordB.col:
            # Case 2: Same col
            newA = self.keyMatrix[(coordA.row + 1) % 5][coordA.col]
            newB = self.keyMatrix[(coordB.row + 1) % 5][coordB.col]
        else:
            # Case 3: Rectangle
            newA = self.keyMatrix[coordA.row][coordB.col]
            newB = self.keyMatrix[coordB.row][coordA.col]
        
        return newA + newB

    def decipher(self, digraph):
        a = digraph[0]
        b = digraph[1]
        coordA = self.findCoord(a)
        coordB = self.findCoord(b)
        newA = a
        newB = b
        
        if coordA.row == coordB.row:
            # Case 1: Same row
            newA = self.keyMatrix[coordA.row][(coordA.col - 1) % 5]
            newB = self.keyMatrix[coordB.row][(coordB.col - 1) % 5]
                    
        elif coordA.col == coordB.col:
            # Case 2: Same col
            newA = self.keyMatrix[(coordA.row - 1) % 5][coordA.col]
            newB = self.keyMatrix[(coordB.row - 1) % 5][coordB.col]
        else:
            # Case 3: Rectangle
            newA = self.keyMatrix[coordA.row][coordB.col]
            newB = self.keyMatrix[coordB.row][coordA.col]
        
        return newA + newB

    def encipherStr(self, string):
        digraphs = []
        i = 0
        while i < len(string):
            if i + 1 == len(string):
                string += "x"
            elif string[i] == string[i+1]:
                string = string[:i + 1] + "x" + string[i + 1:]
            digraphs.append(string[i:i+2])
            i += 2

        return "".join([self.encipher(digraph) for digraph in digraphs])

    def decipherStr(self, string):
        if len(string) % 2 != 0:
            print("Error: String needs to have an even number of characters")
        digraphs = []
        i = 0
        while i < len(string):
            digraphs.append(string[i:i+2])
            i += 2
        return "".join([self.decipher(digraph) for digraph in digraphs])
        
PF = Playfair("abcdefghijklmnopqrstuvwxy")
PF.printMatrix()
print()
val = "sx"
enciphered = PF.encipher(val)
deciphered = PF.decipher(enciphered)
print(val)
print(enciphered)
print(deciphered)

print()
enc = PF.encipherStr("acomma")
dec = PF.decipherStr(enc)
print("Enc: {}".format(enc))
print("Dec: {}".format(dec))

['a', 'b', 'c', 'd', 'e']
['f', 'g', 'h', 'i', 'j']
['k', 'l', 'm', 'n', 'o']
['p', 'q', 'r', 's', 't']
['u', 'v', 'w', 'x', 'y']

sx
xd
sx

Enc: bdknkc
Dec: acomma


In [5]:
stringToCrack = "IVEWNFUILTVWGCGKDGWBUTUGVPLXKCBCAGVPUFZXUIUFVDZXVMMFNRXVLGAUECNQZXVPHBAVBNAQLXAUSPIAVWCXWUFEXWQVLMLQDFIVQEPAPWVQLXZTLSFYFDGAGYEFCGAUWSXWXVBTICAUXWVGRGWPLXAUPAPVLCGTTVRNYZINCWIWAUUFZXUIUFGWBZTZMWLQDFIVPWLRLOUGLGVEABPCTVACFGAIFBXLHFNBVTQAGTTARLQTDFIVPWLQGWXCWDAXQOSBGWIVMWWXFDQVUFGDLOEVWXEWXNVQEVFVKESBINUFVWCLXWOLLORLQTDFIVEWXLVGGFDTFGWGTDVEXLABTLAIUFUAWEAXGVVWGFINOLUFSIXIEURBFNVEAXTABLCSGTUDRYLQDFIVEWXLUFXWGFDTPSGCGTTDVEUFFEAUKTCGAIAVAXGVNFNAINLOCGWBRAXWSBGWIVMWWXFDBLCWVWCRQVGLGAUFXWUFXAGTTAOLCXAUCGWBRAXWSBGWIVMWWXFDWGTDVEAIQGKSRLQTDFGAUFXWWGTDVEPARYLOUGLGVENFDUQVEFGCBGRLQTDFUFUWWFDTIVMELZKAFPPAOGVGCSIWWXWEWKLXUFVWOGOLGZWTDTLGBWZXWGPVVXLGYZPSIWDCFGABZIFPRLQTDFHXUFTATVUTUFFEUFXWVDIWRNDTLBUFFEECNQZXVPHBAVXLAXRVYQVELBCMLQDFIVKAUATVQVLGBQNQGWIVMEABINWIVPDVLGQVSVQVEFMNVWWILGVQZXQBBLISLOWIVPBZUFFDRLQTDFPFINVTEDFGAHSBVEHBYIIVPEWXGVRYLQDFXSGTGUVEABDFUTUFXALXXLQLCLAUVFQEEVWBVPUFGVUFGWUGDWWQWFDTGKSAUFIVNBZCCPFNTWLQDFIVPWIBEDAILMIVSNGURZTYLQDFPSSQFSUEWXDFLIGTTVSINSOTNTGUDTZXXLCLABRYLQDFDWCGAUATXAVTWXMNWKLXIVMEAUWCIVABKTQMGHYMRLQTDFIVPQQBQGDGKAFPGKRLQTDFLNXWXLVTVDBWMWLQDFIVMVTAUFFPQVLCBQUFGVGTLCZXWQHWYITVANVEIWFPMNVWVPFOFDTVGWCNBHXVLGABLSIDEFSIRISPYTFGWPLXFPVWLGAIUTUFXASPYTFGRSXWGUABLSLQDFOLUTUFTWIDEFSIQVEFVPRNEWDWUGQREURLQTDFXLDWWQRGWIUEWQHWRLQTDFAUXNUEQGGSYIRLQTDFIVEDGVGFLIBQDVHDUGBEIVQEQWCUVBCGRLQTDFIVQVLPLOEWKYVDIWXNSXSIUIFPWGFCLOYFFSDUOIQBUFQVUFTVTAUTUFGWABQNLGEGWPHWRLQTDFIVEWPSGCUGFDVEWDHWBCLQUFTVIVPAINCGIEUAGXHVVEIHUGSBXCGXWXLCTZZCQVUTUFXWGWAUXNGWRLQTDFAUWIAPRLQTDFUFXWGWAUXNGWRLQTDFSAUFECNQZXVPRSBQUFLBBLVQLORBBWMEAIXFTVLONPXLDTRLQTDFIVMEAIGFAIAVLGAIQEQVUFGWIBMEAIGFCLAUKYWENSAVZIFPIVLXAUWQHWAUUFZXUIUFTWZXVMAIAVKGGFGCXCIBUIEMLXRNSPGUKCHIWTIEIBIPUTUFGWVWXCSGQVGLLXRLQTDFIWAGGQVFSIVERLQTDFGKLOKCVRVIRTRLGZEMLXASIWVGAVQXEGXWQVXFEUASIWIHWBAPBLSXXVSXVELXXGBQXGWIAPAUYIZXVMAXFVOCSNFURLQTDFIVEDZXVMWBUFXWNFDUFGRLQTDFHXUFVDZXVMWENAWGAIQOSBWIVPDTOLUTXLUFSIQZHBLMLQDFIWAUATFLYGWXDVYIZXVMRBUFAXFNVMLXAIQOSBRGWIAPUFFERLQTDFXLABPCTVCMLQDFFGDWWIVCHWWIKCCGXWKYVRDGNQVTDFVIXGMENXGUPSYMFEWGIWIBKAGUBZEXFUDGNQVTKTCGQVFGLOKGSQNRTWLQDFBCLQIHATMLFEBNIPIBKAUAWELBUGVSSPAUMEBRTWLQDFHXUFWEVGAVQRGTGVIFVTQVUTUFGVAUXLVMGYIVTWQVUGRNHAVRAUAIPELOVWYCNRCGAUYFIVVGVCAUISECIWRNQBEXFUAIUTUFGDIWWGINXWYIWEXLAXGFVESIXLUFGAQRXWUFRYLQDFIVKAVWVPUFTWIDEFSIDVWXUFXVOZCURVNFNAINRNTZRNOIVBWQYAVRXWGVLXFNVMLXRNDTGFDTPSECIWAUIVMEAUWGRLVPDFUGYAVRXWGVLXFNOFEUFGDWPNGVXWFVZCAUIVQVVGLXAUCQIVQVGFILLXAUIVEWOLUFXWDFLSLQDFUFGVFGDWYIXAKYFPWGFCQVGFGFWIYTSBZXCIFWLMLQDFHXGLBRVQWUAVSBRGWGIWFSQZOZIVQEQVEFGCVWRLQTDFRZTYLQVWYCNRPGVTLGRZIMXVMNVAXWRHWESIEVVERLQTDFYALZXAVBLFVBPWLBUFTDHXGFVGAVCQGUFDIVWULMLQDFLOVFVPRLQTDFYAZBWEVAXWPZVRVIEFOTVCAIGFECIWXLAXWGIWFSQU"

NUM_SUBSTRING_LENGTH = 12
THRESHOLD_FREQ = 3

import sys
##################################################################################
##################################################################################
def LOG(message = "\n"):
    sys.stdout.write(message)

##################################################################################
##################################################################################
# Determine frequency analysis of characters
def findNSeq(n, string):
    if n == 0:
        return []
    
    histogram = dict()
    for i in range(0, len(string) - n + 1):
        a = string[i:i+n]
        if a not in histogram:
            histogram[a] = 1
        else:
            histogram[a] = histogram[a] + 1
    return histogram

##################################################################################
##################################################################################
def returnSortedTuples(freq):
    tupleList = [(a, freq[a]) for a in freq]
    tupleList.sort(key = lambda a: a[1], reverse = True)
    return tupleList

##################################################################################
##################################################################################
def dumpList(listToDump):
    count = 0
    for a in listToDump:
        if a[1] > THRESHOLD_FREQ:
#             LOG("{} ({}): {:3d} |  ".format(a[0], decipher(a[0]), a[1]))
            LOG("{}: {:3d} |  ".format(a[0], a[1]))
            count = count + 1
            if count % 5 == 0:
                LOG()
    LOG()

##################################################################################
def main():
    listFreq = [findNSeq(i, stringToCrack) for i in range(1, NUM_SUBSTRING_LENGTH)]
    sortedListFreq = [returnSortedTuples(freq) for freq in listFreq]
    for sortedList in sortedListFreq:
        dumpList(sortedList)

##################################################################################
main()


V: 219 |  F: 203 |  W: 188 |  G: 177 |  L: 162 |  
I: 150 |  U: 146 |  X: 142 |  A: 139 |  Q: 126 |  
T: 117 |  D: 109 |  E:  97 |  B:  84 |  R:  77 |  
C:  74 |  P:  67 |  N:  66 |  S:  65 |  Z:  44 |  
M:  43 |  Y:  38 |  O:  35 |  K:  27 |  H:  27 |  

UF:  59 |  DF:  49 |  LQ:  48 |  IV:  43 |  AU:  38 |  
TD:  32 |  XW:  31 |  FG:  29 |  RL:  28 |  VE:  28 |  
QV:  27 |  GW:  27 |  QT:  25 |  WI:  22 |  FI:  22 |  
GV:  21 |  WG:  20 |  AI:  20 |  LX:  20 |  WX:  20 |  
VP:  20 |  VM:  19 |  GA:  19 |  IW:  19 |  QD:  19 |  
XL:  18 |  ZX:  18 |  VW:  18 |  GF:  17 |  AV:  17 |  
LG:  16 |  FD:  16 |  XV:  16 |  FX:  15 |  VG:  15 |  
VQ:  15 |  SI:  15 |  FV:  14 |  LO:  14 |  EW:  14 |  
XA:  14 |  CG:  14 |  TV:  14 |  VL:  13 |  AX:  13 |  
XF:  12 |  GT:  12 |  AB:  12 |  WW:  12 |  EA:  12 |  
DT:  12 |  FU:  11 |  TU:  11 |  VT:  11 |  FP:  11 |  
UA:  11 |  FE:  11 |  FS:  11 |  ML:  11 |  UG:  11 |  
UT:  11 |  SB:  11 |  UI:  10 |  XG:  10 |  IN:  10 |  
WL:  10 |  BL:  

In [ ]:
# Tetragraph scoring

class Score:
    def __init__(self):
        

In [6]:
# print(stringToCrack)
PF = Playfair("DPQMKEGXTUVNOLZAIBFHWSRCY")
PF.printMatrix()
# print(PF.decipherStr(stringToCrack).lower())
print(PF.decipherStr("RLQTDF").lower())

['D', 'P', 'Q', 'M', 'K']
['E', 'G', 'X', 'T', 'U']
['V', 'N', 'O', 'L', 'Z']
['A', 'I', 'B', 'F', 'H']
['W', 'S', 'R', 'C', 'Y']
comxma
